## Introduction to the Humble Nishiyama Randomness Game
Suppose we have a standard deck of poker cards. For the game, we will be drawing cards, and we keep track of the color of the card.

2 players each choose a sequence of 3 colors, Black or Red. We can choose any combi, i.e. RBR, BRR, RBB etc. for a total of 8 possible combinations. For purposes of illustration, suppose Player 1 chooses RRR, and Player 2 chooses BBB. 

**Version 1 - First Occurrence Rule:**
- Continuously draw cards until a RRR or BBB sequence occurs. Whoever draws their said sequence first gains 1 point.
- For example, if the sequence drawn is RBBRRBRBRRR, player 1 will gain a point.
- Repeat the process for 1000 rounds, and whoever has the highest points win.

**Version 2 - Full Deck Rule:**
- As per version 1, continuously draw cards until a RRR or BBB sequence is observed. 
- If RRR sequence appears, player 1 earns 1 trick, and we discard all cards previously drawn. Likewise, if a BBB sequence appears, player 2 earns 1 trick, and we discard all cards previously drawn. 
- Continue to draw and look for RRR and BBB sequence until all 52 cards have been drawn. Whoever has the most tricks at the end of the round will win 1 point (if both players have the same number of tricks, it is considered a draw). 
- Repeat the process for for 1000 rounds, and whoever has the most points wins.

## Creating Card and Deck classes


In [1]:
from random import shuffle

class Card:
    def __init__(self, rank, suit): 
        specialCards = {11: 'J', 12: 'Q', 13: 'K', 1: 'A'}
        rank_name = specialCards[rank] if rank in specialCards else rank
        
        self.rank = rank_name
        self.suit = suit
        if self.suit in ['S', 'C']:
            self.color = 'B'
        else:
            self.color = 'R'
        
    def __str__(self): 
        return '{}{}'.format(self.rank, self.suit)

class Deck:
    def __init__(self): 
        self.card_list = [Card(rank, suit) for suit in ['S', 'H', 'D', 'C'] for rank in range(1, 14)]
        self.length = len(self.card_list)
    
    def shuffle_cards(self): 
        return shuffle(self.card_list)
    
    def show_all_cards(self):
        for card in self.card_list:
            print(card)
    
    def draw(self):
        self.length -= 1
        return self.card_list.pop()

## Game functions
The following functions will generate the process of the game. They perform the following:
- first_occurrence(player_1, player_2): Used for **Version 1** of the game. Stops drawing a card when the first instance of the player 1 or player 2's chosen sequence appears.
- full_deck(): Used for **Version 2** of the game. Creates a sequence of 52 draws.
- allocate_tricks(sequence, player_1, player_2): Used for **Version 2** of the game. After a sequence is drawn using full_deck(), we go through the sequence until the first instance of player 1 or player 2's chosen sequence appear. We discard all cards previously drawn, and repeat the process until all 52 cards have been exhausted.
- game(player_1, player_2, num_iter, first_instance): Inputs for player 1 and player 2's chosen sequence, number of iterations to play over, and determines which version of the game we play. ***This function will be called to play the game***.

In [2]:
def first_occurrence(player_1, player_2):
    deck = Deck()
    deck.shuffle_cards()
    sequence = []
    drawn_card_history = []
    
    combined = [player_1, player_2]
    
    # Check if player_1 or player_2 sequence has been drawn, and check if all cards has been drawn
    while not any(x in ''.join(sequence) for x in combined) and deck.length > 0:
        drawn_card = deck.draw()
        sequence.append(drawn_card.color)
        drawn_card_history.append(drawn_card.__str__())

    return sequence, drawn_card_history
    
def full_deck():
    deck = Deck()
    deck.shuffle_cards()
    sequence = []
    drawn_card_history = []
    
    # Since we are interested in the entire sequence of 52 cards drawn, we draw all 52 cards
    while deck.length > 0:
        drawn_card = deck.draw()
        sequence.append(drawn_card.color)
        drawn_card_history.append(drawn_card.__str__())

    return sequence, drawn_card_history
    
def allocate_tricks(sequence, player_1, player_2):
    player_1_count, player_2_count = 0, 0
    
    # find the first instance where the sequence chosen by player 1 and player 2 first appears
    index_1, index_2 = sequence.find(player_1), sequence.find(player_2)
    
    # if index = -1, means the player 1 and player 2's sequence no longer appears in the drawn sequences
    while index_1 != -1 and index_2 != -1:
        
        # Smaller index means the sequence is drawn and appears first. Hence, the trick will belong to the player with a smaller index
        winner = min(index_1, index_2)
        if index_1 < index_2:
            player_1_count += 1
        else:
            player_2_count += 1
        
        # Remove all draws prior to the winner's sequence, and repeat the process in the while loop
        sequence = sequence[winner + 3:]
        
        index_1, index_2 = sequence.find(player_1), sequence.find(player_2)
    return player_1_count, player_2_count

def game(player_1, player_2, num_iter, first_instance):
    player_1_win, player_2_win, draw = 0, 0, 0
    
    # Version 1 of the game
    if first_instance == True:
        for i in range(num_iter):
            sequence, drawn_card_history = first_occurrence(player_1, player_2)
            sequence = ''.join(sequence)

            if player_1 in sequence:
                player_1_win += 1
            elif player_2 in sequence:
                player_2_win += 1
            else:
                draw += 1
    
    # Version 2 of the game
    else:
        for i in range(num_iter):
            sequence, drawn_card_history = full_deck()
            sequence = ''.join(sequence)
            player_1_count, player_2_count = allocate_tricks(sequence, player_1, player_2)
            if player_1_count > player_2_count:
                player_1_win += 1
            elif player_1_count < player_2_count:
                player_2_win += 1
            else:
                draw += 1
        
    return 'Player 1 wins: {}, Player 2 wins: {}, Draw: {}'.format(player_1_win, player_2_win, draw)

## Trick to winning the game for Player 2
For this section, we assume that player 2 chooses his sequence after player 1. Because of the non-transitive properties of the game, player 2 always has the advantage of choosing a sequence in response to player 1's sequence.

The strategy for Player 2 is as follows:
- Take the second color from player 1's chosen sequence, and choose the opposite color. This will form the first color of player 2's sequence
- Take the first and second color from player 1's chosen color, and append it to the end of player 2's first color, making it the second and third color respectively.

The following examples will illustrate this example (format as such: Player 1 sequence --> Player 2 sequence):
- BBB --> RBB
- BBR --> RBB
- BRB --> BBR
- BRR --> BBR
- RBB --> RRB
- RBR --> RRB
- RRB --> BRR
- RRR --> BRR

In [3]:
player_1 = 'BBB'
player_2 = 'RBB'

game(player_1, player_2, 1000, first_instance = True)

'Player 1 wins: 118, Player 2 wins: 882, Draw: 0'

In [4]:
player_1 = 'BBR'
player_2 = 'RBB'

game(player_1, player_2, 1000, first_instance = True)

'Player 1 wins: 235, Player 2 wins: 765, Draw: 0'

In [5]:
player_1 = 'BRB'
player_2 = 'BBR'

game(player_1, player_2, 1000, first_instance = True)

'Player 1 wins: 328, Player 2 wins: 672, Draw: 0'

In [6]:
player_1 = 'BRR'
player_2 = 'BBR'

game(player_1, player_2, 1000, first_instance = True)

'Player 1 wins: 351, Player 2 wins: 649, Draw: 0'

In [7]:
player_1 = 'RBB'
player_2 = 'RRB'

game(player_1, player_2, 1000, first_instance = True)

'Player 1 wins: 294, Player 2 wins: 706, Draw: 0'

In [8]:
player_1 = 'RBR'
player_2 = 'RRB'

game(player_1, player_2, 1000, first_instance = True)

'Player 1 wins: 340, Player 2 wins: 660, Draw: 0'

In [9]:
player_1 = 'RRB'
player_2 = 'BRR'

game(player_1, player_2, 1000, first_instance = True)

'Player 1 wins: 240, Player 2 wins: 760, Draw: 0'

In [10]:
player_1 = 'RRR'
player_2 = 'BRR'

game(player_1, player_2, 1000, first_instance = True)

'Player 1 wins: 113, Player 2 wins: 887, Draw: 0'